# Librerias

In [1]:
!pip install tqdm
!pip install tqdm --upgrade

     |████████████████████████████████| 71kB 1.4MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
!pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-09p7h_43
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-09p7h_43
ERROR: Operation cancelled by user


In [3]:
import pandas as pd
import numpy as np
import re
import yake
from tqdm import tqdm

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importamos los datos

In [5]:
#data_tab = pd.read_csv('/content/drive/My Drive/TFM/data/filtered_df_with_imgNames.csv',sep=',',header= 0)
data_tab = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/datos_antiguos/filtered_df_with_imgNames.csv',sep=',',header= 0)

data_tab.tail(3)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CODE,TITLE,DESCRIPTION,DURATION,COLOR,YEAR,DIRECTOR,BUDGET,CWG,RATING,VOTES,METASCORE RATING,GENRES,LANGUAGE,COUNTRY,CONTENT RATING,KEYWORDS,WRITERS,OPENING WEEKEND
107178,tt0051601.png,Fantasmas en la casa,Add a Plot Ã‚Â»,1h 20min,Black and White,(1961),Pedro L. RamÃƒÂ­rez,unknown,unknown,"5,1",17,unknown,Comedy,Spanish,Spain,unknown,based on play | See All (1) Ã‚Â»,Enrique Jardiel Poncela (play) (as Jardiel Pon...,unknown
107179,tt0151309.png,Laughing Boy,Is having fun and saying what you really think...,1h 22min,Color,(2000),Joe Grisaffi,unknown,unknown,"5,5",141,unknown,Comedy,English,USA,unknown,party | part live action | part animation | en...,"George Douglas Lee (screenplay), Joe Grisaffi ...",unknown
107180,tt0022550.png,El puente de Waterloo,A prostitute's self-loathing makes her relucta...,1h 21min,Black and White,(1931),James Whale,"$251,289",unknown,"7,5",2.35,unknown,Drama | Romance,English,USA,unknown,bridge | zeppelin | world war one | air raid |...,"Robert E. Sherwood (from the stage play by), B...",unknown


In [ ]:
data_tab.head(3)

# Preprocesamiento de los datos tabulares

### Limpieza básica

In [ ]:
# Funcion para limpiar la columna keywords
def Clean_keywords(keyword): 

    if re.search(',? See All', keyword): 
  
        pos = re.search(',? See All', keyword).start() 
        return keyword[:pos] 

    if re.search('See All.', keyword): 
  
        pos = re.search('See All.', keyword).start() 
        return 'unknown' 
  
    else: 
        return keyword 

In [ ]:
categorical_vars = ['COLOR','DIRECTOR','GENRES','LANGUAGE','COUNTRY','KEYWORDS','WRITERS']

# Función para cambiar barras por comas
def replace_bar(categorical_vars,data):
  for column in categorical_vars:
    data[column] = data[column].apply(lambda x: x.replace('|',','))

# Función para cambiar espacios en blanco por nada
def replace_space(categorical_vars,data):
  for column in categorical_vars:
    data[column] = data[column].apply(lambda x: x.replace(' ',''))


In [ ]:
data_tab['KEYWORDS'] = data_tab['KEYWORDS'].apply(Clean_keywords)
data_tab['KEYWORDS'] = data_tab['KEYWORDS'].str.replace(', See All (1) Ã‚Â»',' ')

data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].astype(str)
data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].apply(lambda x: x.replace('|',''))
data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].str.replace('See all certifications Ã‚Â»','')
data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].str.replace(' ', '')


replace_bar(categorical_vars,data_tab)
replace_space(categorical_vars,data_tab)

data_tab['YEAR'] = data_tab['YEAR'].str.replace('(','')
data_tab['YEAR'] = data_tab['YEAR'].str.replace(')','')


# Quitamos el .png del código de cada imagen y el tt
data_tab['CODE'] = data_tab['CODE'].str[:-4]

data_tab.head(3)

Limpiamos la columna writers la información que viene entre los paréntesis de las películas

In [ ]:
data_tab['WRITERS'] = data_tab['WRITERS'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

### Convertimos la duración de las películas a minutos


In [ ]:
duration = data_tab['DURATION']
duration_mod = data_tab['DURATION'].str.split('h')

In [ ]:
for i in duration_mod:
  if len(i) == 1:
    i.insert(0, '0')  
  i[1] = i[1][:-3]
  if i[1] == '':
    i[1] = '0'
  elif i[0] != '0':
    i[1] = i[1][1:]
  minutos = int(i[0]) * 60 + int(i[1])
  i.clear()
  i.append(minutos)

In [ ]:
#for i in range(len(duration)):
    #print(duration[i],duration_mod[i])

In [ ]:
for i in range (0,len(duration_mod)):
  duration_mod[i] = duration_mod[i][0]

In [ ]:
data_tab['DURATION'] = duration_mod

In [ ]:
data_tab.head(5)

### Convertimos la columna CWG a euros

In [ ]:
cwg = data_tab['CWG'].str.replace(',','')

In [ ]:
INR = 0.013
GBP = 1.15
NPR = 0.0083

In [ ]:
cwgbn = []
for i in range (0,len(cwg)):
    if cwg[i][:3] == 'INR':
        a= int(cwg[i][3:])*INR
        cwgbn.append(a)
    if cwg[i][:3] == 'GBP':
        a= int(cwg[i][3:])*GBP
        cwgbn.append(a)
    if cwg[i][:3] == 'NPR':
        a= int(cwg[i][3:])*NPR
        cwgbn.append(a)
    if cwg[i][:1] == '$':
        a= int(cwg[i][1:])
        cwgbn.append(a)
    if cwg[i] == 'unknown':
        cwgbn.append("unknown")


In [ ]:
#cwgbn

In [ ]:
data_tab['CWG'] = cwgbn

In [ ]:
data_tab.head(3)

### Convertimos la columna BUDGET a euros
 

In [ ]:
budget= data_tab['BUDGET'].str.replace(',','')

In [ ]:
budget_div = [None]*len(budget)
for i in range (0,len(budget)):
    budget_div[i] = budget[i][:3]

In [ ]:
#set(budget_div)

In [ ]:
currencies_dict={'AED' : 0.27,
'AMD' : 0.0020,
'ARS' : 0.016,
'ATS' : 0.000036,
'AUD' : 0.58,
'AZM' : 0.59,
'BAM' : 0.55,
'BDT' : 0.012, 
'BEF' : 0.0266,
'BGL' : 0.51,
'BND' : 0.64,
'BRL' : 0.18,
'BYR' : 0.35,
'CAD' : 0.65,
'CHF' : 0.94,
'CLP' : 0.0011,
'CNY' : 0.13,
'COP' : 0.00022,
'CZK' : 0.037,
'DEM' : 0.51,
'DKK' : 0.13,
'DOP' : 0.017,
'DZD' : 0.0075,
'EEK' : 0.06,
'EGP' : 0.058,
'ESP' : 166.386,
'EUR' : 1,
'FIM' : 0.17,
'FRF' : 0.15,
'GBP' : 1.10,
'GEL' : 0.26,
'GTQ' : 0.11,
'HKD' : 0.12,
'HRK' : 0.13,
'HUF' : 0.0028,
'IDR' : 0.000057,
'IEP' : 1.3,
'ILS' : 0.25,
'INR' : 0.012,
'IRR' : 0.000022,
'ISK' : 0.0065,
'ITL' : 0.0005,
'JOD' : 1.28,
'JPY' : 0.0083,
'KES' : 0.0086,
'KRW' : 0.00074,
'LKR' : 0.0048,
'LTL' : 1.30,
'LVL' : 1.42,
'MAD' : 0.092,
'MMK' : 0.00065,
'MVR' : 0.059,
'MXN' : 0.039,
'MYR' : 0.21,
'NAD' : 0.052,
'NGN' : 0.0025,
'NLG' : 0.45,
'NOK' : 0.086,
'NPR' : 0.0075,
'NZD' : 0.54,
'PEN' : 0.27,
'PHP' : 0.018,
'PKR' : 0.0055,
'PLN' : 0.22,
'PTE' : 0.000005,
'PYG' : 0.00014,
'ROL' : 0.21,
'RON' : 0.21,
'RUR' : 0.012,
'RWF' : 0.00097,
'SAR' : 0.24,
'SEK' : 0.091,
'SGD' : 0.63,
'SIT' : 0.00417,
'SKK' : 0.033,
'THB' : 0.028,
'TRL' : 0.14,
'TWD' : 0.030,
'UAH' : 0.032,
'VEB' : 0.09078,
'VND' : 0.000039,
'XAU' : 1485.25,
'XCD' : 0.34,
'ZAR' : 0.052}

In [ ]:
budgetbn = []
for item_budget in budget:  
  for currency in currencies_dict.keys():
      if item_budget[:3] == currency:
        a= float(item_budget[3:])*currencies_dict[currency]
        budgetbn.append(a)
        break
      if item_budget[0] == '$':
        a= float(item_budget[1:])*0.90
        budgetbn.append(a)
        break
      if item_budget == 'unknown':
        budgetbn.append('unknown')
        break
      

In [ ]:
len(budgetbn)

In [ ]:
data_tab['BUDGET'] = budgetbn

In [ ]:
data_tab.head(5)

### ESTABLECEMOS EL NÚMERO DE GENEROS, IDIOMAS Y KEYWORDS QUE QUEREMOS TENER EN CUENTA.

Seleccionamos:
3 Géneros
1 Idioma
3 Keywords 

In [ ]:
data_tab_new = data_tab

def split_per_comma_get_first(data, column, num_variables):

    for i in range(0,num_variables):

      item = data[column].str.split(',').str[i]
      data[column+'_'+str(i)] = item

    return data

data_tab_new = split_per_comma_get_first(data_tab_new, "GENRES", 3)
data_tab_new = split_per_comma_get_first(data_tab_new, "LANGUAGE", 1)
data_tab_new = split_per_comma_get_first(data_tab_new, "KEYWORDS", 3)
data_tab_new = split_per_comma_get_first(data_tab_new, "COUNTRY", 1)
data_tab_new = split_per_comma_get_first(data_tab_new, "WRITERS", 1)

In [ ]:
data_tab = data_tab_new
data_tab.head(3)
print(len(data_tab.COUNTRY.unique()))

### CAMBIAR VALORES NaN EN LAS COLUMNAS MODIFICADAS ANTERIORMENTE (GENEROS, IDIOMAS Y KEYWORDS)

Ejemplo: Si una peli no tiene los 3 géneros y solo tiene dos ponemos el primer genero en la columna GENRES_3

In [ ]:
def fill_extra_columns(mainColumn,AuxiliarColumn,data=data_tab):

  new_values_column = []

  for mainVar,auxVar in zip(data[mainColumn].values,data[AuxiliarColumn].values):

    if isinstance(auxVar, str):
      # Tiene un género asociado
      new_values_column.append(auxVar)
    else:
      # Ponemos el primer genero
      new_values_column.append(mainVar)

  data[AuxiliarColumn] = new_values_column

  return data

data_tab = fill_extra_columns('GENRES_0', 'GENRES_1',data_tab)
data_tab = fill_extra_columns('GENRES_0', 'GENRES_2',data_tab)

data_tab = fill_extra_columns('KEYWORDS_0', 'KEYWORDS_1',data_tab)
data_tab = fill_extra_columns('KEYWORDS_0', 'KEYWORDS_2',data_tab)

data_tab.tail(3)

### Reducción la dimensionalidad

Calculamos el número de directores que hay, como nos sale un número muy elevado solo nos quedamos con los actores que han hecho mas de x películas y al resto le llamamos como "John Doe", es decir, todas las películas que tengas actores que hayan hecho menos de x películas su actor será john doe.

In [ ]:
data_tab.head(2)

#### Número de keywords diferentes por columna

In [ ]:
print(len(data_tab.KEYWORDS_0.unique()),
len(data_tab.KEYWORDS_1.unique()),
len(data_tab.KEYWORDS_2.unique()))

#### Número de géneros diferentes por columna

In [ ]:
print(len(data_tab.GENRES_0.unique()),
len(data_tab.GENRES_1.unique()),
len(data_tab.GENRES_2.unique()))

#### Número de idiomas diferentes por columna

In [ ]:
print(len(data_tab.LANGUAGE_0.unique()))

Número de vairables por columna en total.

In [ ]:
print(len(data_tab.COLOR.unique()),
len(data_tab.DIRECTOR.unique()),
len(data_tab.GENRES.unique()),
len(data_tab.LANGUAGE.unique()),
len(data_tab.COUNTRY.unique()),
len(data_tab.KEYWORDS.unique()),
len(data_tab.WRITERS.unique()))

Función que te devuelve los X items que más aparecen en una columna del dataframe

In [ ]:
def listWithTheBest(data_tab,columnName,number=3):

  data =  data_tab[columnName].value_counts()>number
  data = data.reset_index()

  final_list = data.loc[data[columnName]==True]
  final_list = final_list['index'].values
  return final_list

Función que actualiza el dataframe poniendo en los items que aparecen menos de X veces un nombre común para todos ellos

In [ ]:
def new_valuesInDataframe(data_tab,listWithTeBest,columnName,nuevoNombre):
  for i, row in data_tab[columnName].iteritems():
    if row not in listWithTeBest:
      data_tab.at[i,columnName]=nuevoNombre
  return data_tab

In [ ]:
data_tab.head(3)

In [ ]:
best_writers = listWithTheBest(data_tab,'WRITERS_0',3)
data_tab = new_valuesInDataframe(data_tab,best_writers,'WRITERS_0',"JaneDoe")
best_directors = listWithTheBest(data_tab,'DIRECTOR',3)
data_tab = new_valuesInDataframe(data_tab,best_directors,'DIRECTOR',"JohnDoe")

In [ ]:
print(len(data_tab.COLOR.unique()),
len(data_tab.DIRECTOR.unique()),
len(data_tab.LANGUAGE.unique()),
len(data_tab.COUNTRY.unique()),
len(data_tab.WRITERS.unique()))

In [ ]:
data_tab.WRITERS.value_counts()

In [ ]:
data_tab.head(2)

In [ ]:
data_tab['KEYWORDS_1'] = data_tab['KEYWORDS_1'].astype(str)
data_tab['KEYWORDS_2'] = data_tab['KEYWORDS_2'].astype(str)

### Estadarización de la variable CONTENT RATING

In [ ]:
data_tab.loc[data_tab['CONTENT RATING'].str.contains('T'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('A'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('R'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('PG'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('TP'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('X'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('GP'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('M'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('S'),'CONTENT RATING'] = '18'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('M'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('S'),'CONTENT RATING'] = '18'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('NC'),'CONTENT RATING'] = '17'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('G'),'CONTENT RATING'] = '0'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('17'),'CONTENT RATING'] = '17'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('7'),'CONTENT RATING'] = '7'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('12'),'CONTENT RATING'] = '12'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('18'),'CONTENT RATING'] = '18'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('16'),'CONTENT RATING'] = '16'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('Unrated'),'CONTENT RATING'] = '0'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('Passed'),'CONTENT RATING'] = '0'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('Banned'),'CONTENT RATING'] = '18'
data_tab.loc[data_tab['CONTENT RATING'].str.contains('unknown'),'CONTENT RATING'] = '0'
data_tab.loc[data_tab['CONTENT RATING']=='','CONTENT RATING'] = '0'
#data_tab.loc[data_tab['CONTENT RATING'].str.contains(''),'CONTENT RATING'] = '0'
#data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].apply(lambda x: x.replace('','0'))
#data_tab.loc[data_tab['CONTENT RATING'].str.contains('')] = '0'                
data_tab.head(3)

In [ ]:
data_tab['CONTENT RATING'].value_counts()

Categoria 1: Lo puede ver todo el mundo

Categoría 2: Solo mayores de 13

Categoría 3: Lo puden ver solo mayores de 18

In [ ]:
data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].astype(int)

data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].apply(lambda x: 1 if x<13 else x)
data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].apply(lambda x: 2 if x>=13 and x<18 else x)
data_tab['CONTENT RATING'] = data_tab['CONTENT RATING'].apply(lambda x: 3 if x>=18 else x)
data_tab.head(3)

In [ ]:
data_tab.rename(columns={'CONTENT RATING':'CONTENT_RATING'}, inplace=True)

## Transformamos las descripciones a keywords

In [ ]:
data_tab['DESCRIPTION'][data_tab['DESCRIPTION']=='Add a Plot Ã‚Â»'] = 'nodescription'

In [ ]:
descripciones = data_tab.DESCRIPTION.values

In [ ]:
language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 1

keywords_total = []
for i in tqdm(descripciones):
  
  custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
  keywords = custom_kw_extractor.extract_keywords(i)
  keywords_total = keywords_total + [[a_tuple[0] for a_tuple in keywords]]

In [ ]:
keywords_total =np.squeeze(keywords_total)
data_tab['KEYWORDS_DESCRIPTION'] = keywords_total

In [ ]:
data_tab.head(3)

## Guardamos el csv

In [ ]:
data_tab.to_csv("imdb_data.csv",sep=",")



In [ ]:
data_tab